In [1]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [1]:
import argparse

import baal
from baal import ActiveLearningDataset
from baal.active import get_heuristic, ActiveLearningLoop
from baal.bayesian.dropout import MCDropoutModule
from copy import deepcopy

import numpy as np

import segmentation_models_pytorch as smp

from PIL import Image
from pprint import pprint

import torch
from torch import nn
from torch.nn import functional as F

from torchvision.transforms import transforms

from tqdm import tqdm

from typing import List


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# Initialization
%load_ext autoreload
%autoreload 2
import sys
import os
import pathlib

# To be able to reference packages/modules in this repository, this
# relative path must be added to the python path. Your notebook may be 
# in a different folder, so modify this variable to point to the src 
# folder.
proj_notebooks_root = pathlib.Path().absolute()
proj_root_path = proj_notebooks_root.parent
data_path = proj_notebooks_root / "data"

if proj_root_path not in sys.path:
    sys.path.insert(0, proj_root_path.as_posix())
    print("Updated Python Path")

print(f"Project Root Path: {proj_root_path}")
print(f"Project Source Root Path: {proj_notebooks_root}")
print(f"Project Data Path: {data_path}")

Updated Python Path
Project Root Path: /home/default/workspace
Project Source Root Path: /home/default/workspace/ActiveLearning
Project Data Path: /home/default/workspace/ActiveLearning/data


In [3]:

base_dir = data_path / 'ConglomerateConcreteCrackDataset'
if not base_dir.exists():
    dataset_url = 'https://data.lib.vt.edu/ndownloader/articles/16625056/versions/1'

    ! wget {dataset_url} -P {data_path}
    ! unzip -q {data_path / '1'} -d {data_path}
    ! unzip -q {data_path / 'Conglomerate\ Concrete\ Crack\ Detection.zip'} -d {data_path}
    ! mv {data_path/'Conglomerate\ Concrete\ Crack\ Detection'} {base_dir}
    ! mv {data_path / 'README_congl_dataset.rtf'}  {data_path/'ConglomerateConcreteCrackDataset'}     
    ! rm {data_path / 'Conglomerate\ Concrete\ Crack\ Detection.zip'}
    ! rm {data_path / '1'}
else:
    print("Found dataset at ", base_dir.as_posix())

Found dataset at  /home/default/workspace/ActiveLearning/data/ConglomerateConcreteCrackDataset


In [4]:
class FocalLoss(nn.Module):
    """
    References:
        Author: clcarwin
        Site https://github.com/clcarwin/focal_loss_pytorch/blob/master/focalloss.py
    """

    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)):
            self.alpha = torch.Tensor([alpha, 1 - alpha])
        if isinstance(alpha, list):
            self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)  # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))  # N,H*W,C => N*H*W,C
        target = target.view(-1, 1)

        logpt = F.log_softmax(input, dim=1)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)
        pt = logpt.data.exp()

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            select = (target != 0).type(torch.LongTensor).to(self.alpha.device)
            at = self.alpha.gather(0, select.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1 - pt) ** self.gamma * logpt
        if self.size_average:
            return loss.mean()
        else:
            return loss.sum()

In [5]:
def mean_regions(n, grid_size=16):
    # Compute the mean uncertainty per regions.
    # [batch_size, W, H]
    n = torch.from_numpy(n[:, None, ...])
    # [Batch_size, 1, grid, grid]
    out = F.adaptive_avg_pool2d(n, grid_size)
    return np.mean(out.view([-1, grid_size**2]).numpy(), -1)


class ArrayDataset(torch.utils.data.Dataset):
    def __init__(self, array, image_transforms=None, both_transforms=None):
        self.array = array
                
        self.image_transforms = image_transforms
        self.segment_transforms = transforms.Compose([
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.array)

    def __getitem__(self, index):
        imagePath, segmentPath = self.array[index]
        image = Image.open(imagePath).convert('RGB')
        segment = Image.open(segmentPath).convert('L')
        segment = self.segment_transforms(segment)        
            
        if self.image_transforms is not None:
            image = self.image_transforms(image)
        
        # print(image.shape, segment.shape)

        if image.shape != (3, 448, 448):
            print(f"Image shape is {image.shape}")
        if segment.shape != (1, 448, 448):
            print(f"Segment shape is {segment.shape}")
        
        return image, segment.type(torch.int64) 
    
    def split(self, p=0.5):
        count = len(self.array)
        index = np.arange(count)
        first = int(count * p)
        return [
            ArrayDataset(self.array[index[:first]], 
                    image_transforms=self.image_transforms), 
            ArrayDataset(self.array[index[first:]], 
                    image_transforms=self.image_transforms)
        ]

def get_datasets(initial_pool, path):
    transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    X_dir = path/'Train'/'images'
    y_dir = path/'Train'/'masks'

    files = [y for y in X_dir.glob('*')]

    for i in files:
        assert((y_dir / i.name).exists())

    data = np.array([(i, (y_dir / i.name)) for id, i in enumerate(files)])

    dataset = ArrayDataset(data, image_transforms = transform)


    active_set, test_set = dataset.split(0.9)
    print("Active Set: ", len(active_set))
    print("Test Set: ", len(test_set))
    
    active_set = ActiveLearningDataset(active_set)
    
    active_set.label_randomly(initial_pool)
    return active_set, test_set

In [7]:
p_active_learning_steps = 3

p_initial_pool = 60
p_query_size = 60
p_query_interations = 20

p_heuristic = "random"
p_reduce="sum"

p_leaning_epochs=30
p_lr = 0.001
p_batch_size = 16

p_classes = 2

use_cuda = torch.cuda.is_available()
# use_cuda = False
print("Use Cuda:", use_cuda)

active_set, test_set = get_datasets(p_initial_pool, base_dir)

image, segment = active_set[0]
image.shape, segment.shape

Use Cuda: True
Active Set:  8909
Test Set:  990


(torch.Size([3, 448, 448]), torch.Size([1, 448, 448]))

In [13]:

# We will use the FocalLoss
if p_classes > 1:
    criterion = FocalLoss(gamma=2, alpha=0.25)
else:
    criterion = nn.BCEWithLogitsLoss()

# 
# model = smp.Unet(
#     encoder_name="resnext50_32x4d",
#     encoder_depth=5,
#     encoder_weights="imagenet",
#     decoder_use_batchnorm=False,
#     classes=1,
# )
# add_dropout(model, classes=1, activation=None)

model = smp.Unet('resnet34', 
                 classes=p_classes,
                 encoder_weights="imagenet",
                 decoder_use_batchnorm=False)
model.segmentation_head[1] = nn.Dropout2d(0.5)


# This will enable Dropout at test time.
model = MCDropoutModule(model)

# Put everything on GPU.
if use_cuda:
    model.cuda()

# Make an optimizer
optimizer = torch.optim.SGD(
    model.parameters(), 
    lr=p_lr, 
    momentum=0.9, 
    weight_decay=5e-4)

# Keep a copy of the original weights
initial_weights = deepcopy(model.state_dict())

# Add metrics
model = baal.ModelWrapper(model, criterion, replicate_in_memory=False)

In [14]:
heuristic = get_heuristic(p_heuristic, reduction=mean_regions)

# The ALLoop is in charge of predicting the uncertainty and
loop = ActiveLearningLoop(
    active_set,
    model.predict_on_dataset_generator,
    heuristic=heuristic,
    query_size=p_query_size,
    # Instead of predicting on the entire pool, only a subset is used
    max_sample=1000,
    batch_size=p_batch_size,
    iterations=p_query_interations,
    use_cuda=use_cuda,
)
acc = []
for epoch in tqdm(range(p_active_learning_steps)):
    # Following Gal et al. 2016, we reset the weights.
    model.load_state_dict(initial_weights)
    # Train 50 epochs before sampling.
    
    model.train_on_dataset(
        active_set, 
        optimizer, 
        p_batch_size,
        p_leaning_epochs,
        use_cuda,
        workers=12
    )

    # Validation!
    model.test_on_dataset(test_set, p_batch_size, use_cuda)
    should_continue = loop.step()

    logs = model.get_metrics()
    pprint(logs)
    acc.append(logs)
    if not should_continue:
        break

  0%|          | 0/3 [00:00<?, ?it/s]

[802455-MainThread] [baal.modelwrapper:train_on_dataset:83] 2022-12-16T01:37:34.066460Z [info     ] Starting training              dataset=120 epoch=30
[802455-MainThread] [baal.modelwrapper:train_on_dataset:94] 2022-12-16T01:41:38.423057Z [info     ] Training complete              train_loss=0.027370166033506393
[802455-MainThread] [baal.modelwrapper:test_on_dataset:123] 2022-12-16T01:41:38.434731Z [info     ] Starting evaluating            dataset=990
[802455-MainThread] [baal.modelwrapper:test_on_dataset:133] 2022-12-16T01:42:24.514871Z [info     ] Evaluation complete            test_loss=0.02445993572473526
[802455-MainThread] [baal.modelwrapper:predict_on_dataset_generator:232] 2022-12-16T01:42:24.804693Z [info     ] Start Predict                  dataset=1000

100%|██████████| 63/63 [02:02<00:00,  1.95s/it]


 33%|███▎      | 1/3 [06:53<13:46, 413.45s/it]

{'dataset_size': 120,
 'test_loss': 0.02445993572473526,
 'train_loss': 0.027370166033506393}
[802455-MainThread] [baal.modelwrapper:train_on_dataset:83] 2022-12-16T01:44:27.516073Z [info     ] Starting training              dataset=180 epoch=30
[802455-MainThread] [baal.modelwrapper:train_on_dataset:94] 2022-12-16T01:50:00.997024Z [info     ] Training complete              train_loss=0.02446247637271881
[802455-MainThread] [baal.modelwrapper:test_on_dataset:123] 2022-12-16T01:50:01.004492Z [info     ] Starting evaluating            dataset=990
[802455-MainThread] [baal.modelwrapper:test_on_dataset:133] 2022-12-16T01:50:47.317267Z [info     ] Evaluation complete            test_loss=0.024349955841898918
[802455-MainThread] [baal.modelwrapper:predict_on_dataset_generator:232] 2022-12-16T01:50:47.719892Z [info     ] Start Predict                  dataset=1000

100%|██████████| 63/63 [02:02<00:00,  1.95s/it]

 67%|██████▋   | 2/3 [15:16<07:46, 466.24s/it]


{'dataset_size': 180,
 'test_loss': 0.024349955841898918,
 'train_loss': 0.02446247637271881}
[802455-MainThread] [baal.modelwrapper:train_on_dataset:83] 2022-12-16T01:52:50.710970Z [info     ] Starting training              dataset=240 epoch=30
[802455-MainThread] [baal.modelwrapper:train_on_dataset:94] 2022-12-16T01:59:28.223262Z [info     ] Training complete              train_loss=0.02128368429839611
[802455-MainThread] [baal.modelwrapper:test_on_dataset:123] 2022-12-16T01:59:28.242017Z [info     ] Starting evaluating            dataset=990
[802455-MainThread] [baal.modelwrapper:test_on_dataset:133] 2022-12-16T02:00:15.817962Z [info     ] Evaluation complete            test_loss=0.024393050000071526
[802455-MainThread] [baal.modelwrapper:predict_on_dataset_generator:232] 2022-12-16T02:00:16.114333Z [info     ] Start Predict                  dataset=1000

100%|██████████| 63/63 [02:02<00:00,  1.95s/it]


100%|██████████| 3/3 [24:45<00:00, 495.02s/it]

{'dataset_size': 240,
 'test_loss': 0.024393050000071526,
 'train_loss': 0.02128368429839611}


In [15]:
acc

[{'test_loss': 0.02445993572473526,
  'train_loss': 0.027370166033506393,
  'dataset_size': 120},
 {'test_loss': 0.024349955841898918,
  'train_loss': 0.02446247637271881,
  'dataset_size': 180},
 {'test_loss': 0.024393050000071526,
  'train_loss': 0.02128368429839611,
  'dataset_size': 240}]